In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from albert import modeling
from albert import optimization
from albert import tokenization
import tensorflow as tf
import numpy as np

In [3]:
# !wget https://f000.backblazeb2.com/file/malaya-model/bert-bahasa/albert-tiny-2020-04-17.tar.gz
# !tar -zxf albert-tiny-2020-04-17.tar.gz

In [4]:
!ls albert-tiny-2020-04-17

config.json				model.ckpt-1000000.meta
model.ckpt-1000000.data-00000-of-00001	sp10m.cased.v10.model
model.ckpt-1000000.index		sp10m.cased.v10.vocab


In [5]:
tokenizer = tokenization.FullTokenizer(
      vocab_file='albert-tiny-2020-04-17/sp10m.cased.v10.vocab', do_lower_case=False,
      spm_model_file='albert-tiny-2020-04-17/sp10m.cased.v10.model')


INFO:tensorflow:loading sentence piece model


In [6]:
albert_config = modeling.AlbertConfig.from_json_file('albert-tiny-2020-04-17/config.json')
albert_config

In [7]:
prefix = 'https://f000.backblazeb2.com/file/malay-dataset/tagging/ontonotes5/'

urls = """
augmentation-address-ontonotes5.json
augmentation-event-ontonotes5.json
augmentation-fac-ontonotes5.json
augmentation-gpe-ontonotes5.json
augmentation-language-ontonotes5.json
augmentation-law-ontonotes5.json
augmentation-loc-ontonotes5.json
augmentation-norp-ontonotes5.json
augmentation-org-ontonotes5.json
augmentation-person-ontonotes5.json
augmentation-product-ontonotes5.json
augmentation-work-of-art-ontonotes5.json
ontonotes5-train-test.json
"""
urls = list(filter(None, urls.split('\n')))

import os

# uncomment to download
# for url in urls:
#     print(url)
#     os.system(f'wget {prefix}{url}')

In [8]:
import json

with open('ontonotes5-train-test.json') as fopen:
    data = json.load(fopen)
data.keys()

dict_keys(['train_X', 'train_Y', 'test_X', 'test_Y'])

In [9]:
train_X = data['train_X']
test_X = data['test_X']
train_Y = data['train_Y']
test_Y = data['test_Y']

In [10]:
d = [
    {'Tag': 'OTHER', 'Description': 'other'},
    {'Tag': 'ADDRESS', 'Description': 'Address of physical location.'},
    {'Tag': 'PERSON', 'Description': 'People, including fictional.'},
    {
        'Tag': 'NORP',
        'Description': 'Nationalities or religious or political groups.',
    },
    {
        'Tag': 'FAC',
        'Description': 'Buildings, airports, highways, bridges, etc.',
    },
    {
        'Tag': 'ORG',
        'Description': 'Companies, agencies, institutions, etc.',
    },
    {'Tag': 'GPE', 'Description': 'Countries, cities, states.'},
    {
        'Tag': 'LOC',
        'Description': 'Non-GPE locations, mountain ranges, bodies of water.',
    },
    {
        'Tag': 'PRODUCT',
        'Description': 'Objects, vehicles, foods, etc. (Not services.)',
    },
    {
        'Tag': 'EVENT',
        'Description': 'Named hurricanes, battles, wars, sports events, etc.',
    },
    {'Tag': 'WORK_OF_ART', 'Description': 'Titles of books, songs, etc.'},
    {'Tag': 'LAW', 'Description': 'Named documents made into laws.'},
    {'Tag': 'LANGUAGE', 'Description': 'Any named language.'},
    {
        'Tag': 'DATE',
        'Description': 'Absolute or relative dates or periods.',
    },
    {'Tag': 'TIME', 'Description': 'Times smaller than a day.'},
    {'Tag': 'PERCENT', 'Description': 'Percentage, including "%".'},
    {'Tag': 'MONEY', 'Description': 'Monetary values, including unit.'},
    {
        'Tag': 'QUANTITY',
        'Description': 'Measurements, as of weight or distance.',
    },
    {'Tag': 'ORDINAL', 'Description': '"first", "second", etc.'},
    {
        'Tag': 'CARDINAL',
        'Description': 'Numerals that do not fall under another type.',
    },
]
d = [d['Tag'] for d in d]
d = ['PAD', 'X'] + d
tag2idx = {i: no for no, i in enumerate(d)}
idx2tag = {no: i for no, i in enumerate(d)}

In [11]:
from glob import glob

augmented = glob('augmentation-*-ontonotes5.json')

for f in augmented:
    print(f)
    with open(f) as fopen:
        data = json.load(fopen)
        
    print(len(data.get('train_X', [])), len(data.get('train_Y', [])))
    print(len(data.get('test_X', [])), len(data.get('test_Y', [])))
    
    train_X.extend(data.get('train_X', []))
    train_Y.extend(data.get('train_Y', []))
    test_X.extend(data.get('test_X', []))
    test_Y.extend(data.get('test_Y', []))

augmentation-org-ontonotes5.json
35150 35150
6300 6300
augmentation-fac-ontonotes5.json
17040 17040
4260 4260
augmentation-loc-ontonotes5.json
13040 13040
2460 2460
augmentation-gpe-ontonotes5.json
21060 21060
0 0
augmentation-work-of-art-ontonotes5.json
4020 4020
1020 1020
augmentation-event-ontonotes5.json
1665 1665
0 0
augmentation-person-ontonotes5.json
37883 37883
2530 2530
augmentation-product-ontonotes5.json
7040 7040
1760 1760
augmentation-law-ontonotes5.json
12584 12584
3161 3161
augmentation-language-ontonotes5.json
6860 6860
0 0
augmentation-norp-ontonotes5.json
30660 30660
5940 5940
augmentation-address-ontonotes5.json
57502 57502
15106 15106


In [12]:
from tqdm import tqdm

def XY(strings):
    left_train, right_train = strings[0], strings[1]
    X, Y, MASK = [], [], []
    for i in tqdm(range(len(left_train))):
        left = [d for d in left_train[i]]
        right = [d for d in right_train[i]]
        bert_tokens = ['[CLS]']
        y = ['PAD']
        for no, orig_token in enumerate(left):
            t = tokenizer.tokenize(orig_token)
            bert_tokens.extend(t)
            if len(t):
                y.append(right[no])
            y.extend(['X'] * (len(t) - 1))
        bert_tokens.append('[SEP]')
        y.append('PAD')
        x = tokenizer.convert_tokens_to_ids(bert_tokens)
        y = [tag2idx[i] for i in y]
        input_mask = [1] * len(y)
        if len(x) != len(y):
            print(i)
        X.append(x)
        Y.append(y)
        MASK.append(input_mask)
    return [(X, Y, MASK,)]

In [13]:
import cleaning
t = cleaning.multiprocessing(train_X, train_Y, XY)
train_X, train_Y, train_masks = [], [], []
for x, y, m in t:
    train_X.extend(x)
    train_Y.extend(y)
    train_masks.extend(m)

100%|██████████| 40812/40812 [00:32<00:00, 1247.03it/s]


In [14]:
t = cleaning.multiprocessing(test_X, test_Y, XY)
test_X, test_Y, test_masks = [], [], []
for x, y, m in t:
    test_X.extend(x)
    test_Y.extend(y)
    test_masks.extend(m)

100%|██████████| 5494/5494 [00:04<00:00, 1163.81it/s]


In [15]:
from sklearn.utils import shuffle

train_X, train_Y, train_masks = shuffle(train_X, train_Y, train_masks)

In [16]:
BERT_INIT_CHKPNT = 'albert-tiny-2020-04-17/model.ckpt-1000000'

In [17]:
epoch = 5
batch_size = 32
warmup_proportion = 0.1
num_train_steps = int(1000 / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [18]:
def create_initializer(initializer_range=0.02):
    return tf.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True
    ):
        self.X = tf.placeholder(tf.int32, [None, None])
        self.MASK = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.maxlen = tf.shape(self.X)[1]
        self.lengths = tf.count_nonzero(self.X, 1)
        
        model = modeling.AlbertModel(
            config=albert_config,
            is_training=training,
            input_ids=self.X,
            input_mask=self.MASK,
            use_one_hot_embeddings=False)
        output_layer = model.get_sequence_output()
        output_layer = tf.layers.dense(
            output_layer,
            albert_config.hidden_size,
            activation=tf.tanh,
            kernel_initializer=create_initializer())
        logits = tf.layers.dense(output_layer, dimension_output,
                                         kernel_initializer=create_initializer())
        y_t = self.Y
        log_likelihood, transition_params = tf.contrib.crf.crf_log_likelihood(
            logits, y_t, self.lengths
        )
        self.cost = tf.reduce_mean(-log_likelihood)
        self.optimizer = tf.train.AdamOptimizer(
            learning_rate = learning_rate
        ).minimize(self.cost)
        mask = tf.sequence_mask(self.lengths, maxlen = self.maxlen)
        self.tags_seq, tags_score = tf.contrib.crf.crf_decode(
            logits, transition_params, self.lengths
        )
        self.tags_seq = tf.identity(self.tags_seq, name = 'logits')

        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(self.tags_seq, mask)
        mask_label = tf.boolean_mask(y_t, mask)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [19]:
dimension_output = len(tag2idx)
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.global_variables_initializer())
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope = 'bert')
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, BERT_INIT_CHKPNT)

Instructions for updating:
reduction_indices is deprecated, use axis instead




Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
INFO:tensorflow:Restoring parameters from albert-tiny-2020-04-17/model.ckpt-1000000


In [20]:
def merge_sentencepiece_tokens_tagging(x, y):
    new_paired_tokens = []
    n_tokens = len(x)
    rejected = ['[CLS]', '[SEP]']

    i = 0

    while i < n_tokens:

        current_token, current_label = x[i], y[i]
        if not current_token.startswith('▁') and current_token not in rejected:
            previous_token, previous_label = new_paired_tokens.pop()
            merged_token = previous_token
            merged_label = [previous_label]
            while (
                not current_token.startswith('▁')
                and current_token not in rejected
            ):
                merged_token = merged_token + current_token.replace('▁', '')
                merged_label.append(current_label)
                i = i + 1
                current_token, current_label = x[i], y[i]
            merged_label = merged_label[0]
            new_paired_tokens.append((merged_token, merged_label))

        else:
            new_paired_tokens.append((current_token, current_label))
            i = i + 1

    words = [
        i[0].replace('▁', '')
        for i in new_paired_tokens
        if i[0] not in rejected
    ]
    labels = [i[1] for i in new_paired_tokens if i[0] not in rejected]
    return words, labels

In [21]:
string = 'KUALA LUMPUR : Sempena sambutan Aidilfitri minggu depan, Perdana Menteri Tun Dr Mahathir Mohamad dan Menteri Pengangkutan Anthony Loke Siew Fook menitipkan pesanan khas kepada orang ramai yang mahu pulang ke kampung halaman masing-masing. Dalam video pendek terbitan Jabatan Keselamatan Jalan Raya (JKJR) itu, Dr Mahathir menasihati mereka supaya berhenti berehat dan tidur sebentar  sekiranya mengantuk ketika memandu.'

import re

def entities_textcleaning(string, lowering = False):
    """
    use by entities recognition, pos recognition and dependency parsing
    """
    string = re.sub(r'[ ]+', ' ', string).strip()
    original_string = string.split()
    if lowering:
        string = string.lower()
    string = [
        (original_string[no], word.title() if word.isupper() else word)
        for no, word in enumerate(string.split())
        if len(word)
    ]
    return [s[0] for s in string], [s[1] for s in string]

def parse_X(left):
    bert_tokens = ['[CLS]']
    for no, orig_token in enumerate(left):
        t = tokenizer.tokenize(orig_token)
        bert_tokens.extend(t)
    bert_tokens.append("[SEP]")
    input_mask = [1] * len(bert_tokens)
    return tokenizer.convert_tokens_to_ids(bert_tokens), bert_tokens, input_mask

sequence = entities_textcleaning(string)[1]
parsed_sequence, bert_sequence, input_mask = parse_X(sequence)

In [22]:
predicted = sess.run(model.tags_seq,
                feed_dict = {
                    model.X: [parsed_sequence],
                    model.MASK: [input_mask]
                },
        )[0]
merged = merge_sentencepiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
list(zip(merged[0], merged[1]))

[('Kuala', 'ADDRESS'),
 ('Lumpur', 'ADDRESS'),
 (':', 'CARDINAL'),
 ('Sempena', 'PERCENT'),
 ('sambutan', 'TIME'),
 ('Aidilfitri', 'ADDRESS'),
 ('minggu', 'CARDINAL'),
 ('depan,', 'PERCENT'),
 ('Perdana', 'PRODUCT'),
 ('Menteri', 'ADDRESS'),
 ('Tun', 'EVENT'),
 ('Dr', 'ORDINAL'),
 ('Mahathir', 'DATE'),
 ('Mohamad', 'LANGUAGE'),
 ('dan', 'ORDINAL'),
 ('Menteri', 'X'),
 ('Pengangkutan', 'EVENT'),
 ('Anthony', 'FAC'),
 ('Loke', 'OTHER'),
 ('Siew', 'OTHER'),
 ('Fook', 'OTHER'),
 ('menitipkan', 'DATE'),
 ('pesanan', 'LANGUAGE'),
 ('khas', 'X'),
 ('kepada', 'EVENT'),
 ('orang', 'ORDINAL'),
 ('ramai', 'EVENT'),
 ('yang', 'ORDINAL'),
 ('mahu', 'X'),
 ('pulang', 'EVENT'),
 ('ke', 'FAC'),
 ('kampung', 'OTHER'),
 ('halaman', 'LOC'),
 ('masing-masing.', 'CARDINAL'),
 ('Dalam', 'PAD'),
 ('video', 'PAD'),
 ('pendek', 'PAD'),
 ('terbitan', 'PAD'),
 ('Jabatan', 'PAD'),
 ('Keselamatan', 'PAD'),
 ('Jalan', 'PAD'),
 ('Raya', 'OTHER'),
 ('(Jkjr)', 'LOC'),
 ('itu,', 'ORDINAL'),
 ('Dr', 'PAD'),
 ('Mahathir'

In [23]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [24]:
import time

for e in range(epoch):
    lasttime = time.time()
    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_X), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x = train_X[i : index]
        batch_y = train_Y[i : index]
        batch_masks = train_masks[i : index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = pad_sequences(batch_y, padding='post')
        batch_masks = pad_sequences(batch_masks, padding='post')
        
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
                model.MASK: batch_masks,
            },
        )
        assert not np.isnan(cost)
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    pbar = tqdm(
        range(0, len(test_X), batch_size), desc = 'test minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(test_X))
        batch_x = test_X[i : index]
        batch_y = test_Y[i : index]
        batch_masks = test_masks[i : index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = pad_sequences(batch_y, padding='post')
        batch_masks = pad_sequences(batch_masks, padding='post')
        
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.X: batch_x,
                model.Y: batch_y,
                model.MASK: batch_masks,
            },
        )
        assert not np.isnan(cost)
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
    
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)

    print('time taken:', time.time() - lasttime)
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (e, train_loss, train_acc, test_loss, test_acc)
    )
    predicted = sess.run(model.tags_seq,
                feed_dict = {
                    model.X: [parsed_sequence],
                    model.MASK: [input_mask]
                },
        )[0]
    merged = merge_sentencepiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
    print(list(zip(merged[0], merged[1])))

train minibatch loop:   0%|          | 0/20407 [00:00<?, ?it/s]

time taken: 3738.9464395046234
epoch: 0, training loss: 8.857645, training acc: 0.963190, valid loss: 7.090721, valid acc: 0.969520

[('Kuala', 'EVENT'), ('Lumpur', 'GPE'), (':', 'OTHER'), ('Sempena', 'OTHER'), ('sambutan', 'DATE'), ('Aidilfitri', 'EVENT'), ('minggu', 'DATE'), ('depan,', 'DATE'), ('Perdana', 'OTHER'), ('Menteri', 'OTHER'), ('Tun', 'PERSON'), ('Dr', 'PERSON'), ('Mahathir', 'PERSON'), ('Mohamad', 'PERSON'), ('dan', 'OTHER'), ('Menteri', 'OTHER'), ('Pengangkutan', 'OTHER'), ('Anthony', 'PERSON'), ('Loke', 'PERSON'), ('Siew', 'PERSON'), ('Fook', 'PERSON'), ('menitipkan', 'OTHER'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing-masing.', 'OTHER'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'ORG'), ('Keselamatan', 'ORG'), ('Jalan', 'ORG'), ('

train minibatch loop:   0%|          | 0/20407 [00:00<?, ?it/s]

time taken: 3734.162833213806
epoch: 1, training loss: 3.324027, training acc: 0.984087, valid loss: 6.586186, valid acc: 0.973797

[('Kuala', 'EVENT'), ('Lumpur', 'GPE'), (':', 'OTHER'), ('Sempena', 'EVENT'), ('sambutan', 'DATE'), ('Aidilfitri', 'EVENT'), ('minggu', 'DATE'), ('depan,', 'DATE'), ('Perdana', 'OTHER'), ('Menteri', 'OTHER'), ('Tun', 'PERSON'), ('Dr', 'PERSON'), ('Mahathir', 'PERSON'), ('Mohamad', 'PERSON'), ('dan', 'OTHER'), ('Menteri', 'OTHER'), ('Pengangkutan', 'OTHER'), ('Anthony', 'PERSON'), ('Loke', 'PERSON'), ('Siew', 'PERSON'), ('Fook', 'PERSON'), ('menitipkan', 'OTHER'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing-masing.', 'OTHER'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'ORG'), ('Keselamatan', 'ORG'), ('Jalan', 'ORG'), ('R

train minibatch loop:   0%|          | 0/20407 [00:00<?, ?it/s]

time taken: 3736.620117664337
epoch: 2, training loss: 1.960042, training acc: 0.990483, valid loss: 6.897969, valid acc: 0.975134

[('Kuala', 'EVENT'), ('Lumpur', 'EVENT'), (':', 'OTHER'), ('Sempena', 'OTHER'), ('sambutan', 'DATE'), ('Aidilfitri', 'EVENT'), ('minggu', 'DATE'), ('depan,', 'DATE'), ('Perdana', 'OTHER'), ('Menteri', 'ORDINAL'), ('Tun', 'PERSON'), ('Dr', 'PERSON'), ('Mahathir', 'PERSON'), ('Mohamad', 'PERSON'), ('dan', 'OTHER'), ('Menteri', 'OTHER'), ('Pengangkutan', 'OTHER'), ('Anthony', 'PERSON'), ('Loke', 'PERSON'), ('Siew', 'PERSON'), ('Fook', 'PERSON'), ('menitipkan', 'OTHER'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing-masing.', 'OTHER'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'ORG'), ('Keselamatan', 'ORG'), ('Jalan', 'ORG'),

train minibatch loop:   0%|          | 0/20407 [00:00<?, ?it/s]

time taken: 3736.418641090393
epoch: 3, training loss: 1.255660, training acc: 0.993904, valid loss: 7.431076, valid acc: 0.976113

[('Kuala', 'EVENT'), ('Lumpur', 'GPE'), (':', 'OTHER'), ('Sempena', 'OTHER'), ('sambutan', 'DATE'), ('Aidilfitri', 'EVENT'), ('minggu', 'DATE'), ('depan,', 'DATE'), ('Perdana', 'OTHER'), ('Menteri', 'CARDINAL'), ('Tun', 'PERSON'), ('Dr', 'PERSON'), ('Mahathir', 'PERSON'), ('Mohamad', 'PERSON'), ('dan', 'OTHER'), ('Menteri', 'OTHER'), ('Pengangkutan', 'OTHER'), ('Anthony', 'PERSON'), ('Loke', 'PERSON'), ('Siew', 'PERSON'), ('Fook', 'PERSON'), ('menitipkan', 'OTHER'), ('pesanan', 'OTHER'), ('khas', 'OTHER'), ('kepada', 'OTHER'), ('orang', 'OTHER'), ('ramai', 'OTHER'), ('yang', 'OTHER'), ('mahu', 'OTHER'), ('pulang', 'OTHER'), ('ke', 'OTHER'), ('kampung', 'OTHER'), ('halaman', 'OTHER'), ('masing-masing.', 'OTHER'), ('Dalam', 'OTHER'), ('video', 'OTHER'), ('pendek', 'OTHER'), ('terbitan', 'OTHER'), ('Jabatan', 'ORG'), ('Keselamatan', 'ORG'), ('Jalan', 'ORG'), 

train minibatch loop:   0%|          | 65/20407 [00:10<56:43,  5.98it/s, accuracy=0.998, cost=0.6]   


KeyboardInterrupt: 

In [25]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'albert-tiny-entities/model.ckpt')

'albert-tiny-entities/model.ckpt'

In [26]:
dimension_output = len(tag2idx)
learning_rate = 2e-5

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate,
    training = False
)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(tf.trainable_variables())
saver.restore(sess, 'albert-tiny-entities/model.ckpt')

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


INFO:tensorflow:Restoring parameters from albert-tiny-entities/model.ckpt


In [27]:
def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            out_i.append(idx2tag[p])
        out.append(out_i)
    return out

In [28]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_X), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_X))
    batch_x = test_X[i : index]
    batch_y = test_Y[i : index]
    batch_masks = test_masks[i : index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = pad_sequences(batch_y, padding='post')
    batch_masks = pad_sequences(batch_masks, padding='post')
    predicted = pred2label(sess.run(model.tags_seq,
            feed_dict = {
                model.X: batch_x,
                model.MASK: batch_masks,
            },
    ))
    real = pred2label(batch_y)
    predict_Y.extend(predicted)
    real_Y.extend(real)

validation minibatch loop: 100%|██████████| 2748/2748 [05:19<00:00,  8.61it/s]


In [29]:
temp_real_Y = []
for r in real_Y:
    temp_real_Y.extend(r)
    
temp_predict_Y = []
for r in predict_Y:
    temp_predict_Y.extend(r)

In [30]:
from sklearn.metrics import classification_report
print(classification_report(temp_real_Y, temp_predict_Y, digits = 5))

              precision    recall  f1-score   support

     ADDRESS    0.99322   0.99706   0.99513     93446
    CARDINAL    0.89708   0.84719   0.87142     48255
        DATE    0.92805   0.90984   0.91886    126548
       EVENT    0.82915   0.90247   0.86426      5711
         FAC    0.93366   0.84317   0.88611     27392
         GPE    0.90096   0.91216   0.90653    101357
    LANGUAGE    0.87336   0.74720   0.80537       803
         LAW    0.91511   0.91677   0.91594     24834
         LOC    0.90699   0.89105   0.89895     34538
       MONEY    0.83930   0.84377   0.84152     30032
        NORP    0.88694   0.85367   0.86999     57014
     ORDINAL    0.82854   0.88041   0.85369      6213
         ORG    0.91629   0.89916   0.90764    219533
       OTHER    0.98521   0.98937   0.98728   3553350
         PAD    1.00000   1.00000   1.00000   1291289
     PERCENT    0.95564   0.95898   0.95731     21722
      PERSON    0.90445   0.93733   0.92060    101981
     PRODUCT    0.79770   0

In [31]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'bert/embeddings/word_embeddings',
 'bert/embeddings/token_type_embeddings',
 'bert/embeddings/position_embeddings',
 'bert/embeddings/LayerNorm/gamma',
 'bert/encoder/embedding_hidden_mapping_in/kernel',
 'bert/encoder/embedding_hidden_mapping_in/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/query/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/key/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/self/value/bias',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/kernel',
 'bert/encoder/transformer/group_0/inner_group_0/attention_1/output/dense/bias',
 'bert/encoder/transformer/group_0/inner_group_0/Lay

In [32]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [33]:
freeze_graph('albert-tiny-entities', strings)

INFO:tensorflow:Restoring parameters from albert-tiny-entities/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 30 variables.
INFO:tensorflow:Converted 30 variables to const ops.
1523 ops in the final graph.


In [34]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

g = load_graph('albert-tiny-entities/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
mask = g.get_tensor_by_name('import/Placeholder_1:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)

/home/husein/.local/lib/python3.6/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [35]:
string = 'Kyrgios, 25 , membuat pesanan itu kerana menyedaari pelbagai kesukaran menimpa rakyat Australia ekoran perintah kawalan pergerakan yang diumumkan Mac lalu bagi memerangi wabak COVID-19 di negara berkenaan. Pemain tenis ranking ke-40 dunia yang dilahirkan di Canberra itu meminta pengikut dan penyokongnya agar jangan tidur dalam keadaan perut kosong dalam hantaran Instagram yang meraih lebih 92,000 tanda suka.'

ori, sequence = entities_textcleaning(string)
parsed_sequence, bert_sequence, input_mask = parse_X(sequence)

In [36]:
predicted = test_sess.run(logits,
            feed_dict = {
                x: [parsed_sequence],
                mask: [input_mask]
            },
    )[0]
merged = merge_sentencepiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
print(list(zip(merged[0], merged[1])))

[('Kyrgios,', 'OTHER'), ('25', 'OTHER'), (',', 'OTHER'), ('membuat', 'OTHER'), ('pesanan', 'OTHER'), ('itu', 'OTHER'), ('kerana', 'OTHER'), ('menyedaari', 'OTHER'), ('pelbagai', 'OTHER'), ('kesukaran', 'OTHER'), ('menimpa', 'OTHER'), ('rakyat', 'OTHER'), ('Australia', 'NORP'), ('ekoran', 'OTHER'), ('perintah', 'OTHER'), ('kawalan', 'OTHER'), ('pergerakan', 'OTHER'), ('yang', 'OTHER'), ('diumumkan', 'OTHER'), ('Mac', 'DATE'), ('lalu', 'DATE'), ('bagi', 'OTHER'), ('memerangi', 'OTHER'), ('wabak', 'EVENT'), ('Covid-19', 'EVENT'), ('di', 'OTHER'), ('negara', 'OTHER'), ('berkenaan.', 'OTHER'), ('Pemain', 'OTHER'), ('tenis', 'OTHER'), ('ranking', 'OTHER'), ('ke-40', 'ORDINAL'), ('dunia', 'OTHER'), ('yang', 'OTHER'), ('dilahirkan', 'OTHER'), ('di', 'OTHER'), ('Canberra', 'GPE'), ('itu', 'OTHER'), ('meminta', 'OTHER'), ('pengikut', 'OTHER'), ('dan', 'OTHER'), ('penyokongnya', 'OTHER'), ('agar', 'OTHER'), ('jangan', 'OTHER'), ('tidur', 'OTHER'), ('dalam', 'OTHER'), ('keadaan', 'OTHER'), ('perut

In [37]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
tf.set_random_seed(0)

In [38]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'albert-tiny-entities/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

inputs = ['Placeholder', 'Placeholder_1']
outputs = ['dense/BiasAdd']

print(pb, inputs)

transformed_graph_def = TransformGraph(input_graph_def, 
                                       inputs,
                                       ['logits'] + outputs, transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.
albert-tiny-entities/frozen_model.pb ['Placeholder', 'Placeholder_1']
